<a href="https://colab.research.google.com/github/prateekgargX/Analytics-GC-2022/blob/bhavya/analyticsGC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! gdown --id 1E0wZhuDBzfhpVJIg1qdasQ0_Jn6BLmhA

Downloading...
From: https://drive.google.com/uc?id=1E0wZhuDBzfhpVJIg1qdasQ0_Jn6BLmhA
To: /content/data.csv
100% 345k/345k [00:00<00:00, 95.4MB/s]


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
data = pd.read_csv('/content/data.csv')
data

,y,x1,x2,x3,x4
0,0,3861.22,3861.66,3860.75,3861.38
1,0,3861.19,3861.38,3860.16,3861.25
2,0,3861.38,3861.38,3858.47,3858.53
3,1,3858.72,3858.72,3857.03,3858.19
4,1,3858.09,3859.16,3858.03,3858.81
...,...,...,...,...,...
9995,0,3794.75,3795.00,3793.75,3794.88
9996,1,3794.78,3795.09,3794.53,3794.72
9997,0,3794.75,3795.09,3794.47,3794.91
9998,1,3794.94,3795.13,3793.38,3794.34


In [4]:
data['x2'] = data['x2'] - data['x3']
data = data.drop(columns=['x3'])

In [5]:
data

,y,x1,x2,x4
0,0,3861.22,0.91,3861.38
1,0,3861.19,1.22,3861.25
2,0,3861.38,2.91,3858.53
3,1,3858.72,1.69,3858.19
4,1,3858.09,1.13,3858.81
...,...,...,...,...
9995,0,3794.75,1.25,3794.88
9996,1,3794.78,0.56,3794.72
9997,0,3794.75,0.62,3794.91
9998,1,3794.94,1.75,3794.34


In [6]:
data_y = data['y'].values
data_x = data.drop(columns = ['y'])

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(data_x)

data_x = scaler.fit_transform(data_x)
# data_x = data_x.reshape(-1,1,4)

train_x, test_x, train_y, test_y = train_test_split(data_x, data_y, test_size=0.3, shuffle=1, random_state=69)
test_x, val_x, test_y, val_y = train_test_split(test_x, test_y, test_size=0.5, shuffle=1, random_state=69)

In [7]:
print(train_x.shape, train_y.shape, test_x.shape, test_y.shape, val_x.shape, val_y.shape)

(7000, 3) (7000,) (1500, 3) (1500,) (1500, 3) (1500,)


In [8]:
# from tensorflow import keras
from keras.layers import Dense, LSTM, GRU, Dropout
from keras.layers.embeddings import Embedding
from keras.models import Sequential
from tqdm.keras import TqdmCallback
from keras.callbacks import EarlyStopping

es_callback = EarlyStopping(monitor='val_accuracy', patience=5)

import tensorflow as tf
# Silence deprecated-function warnings
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [9]:
model = Sequential([
    # Embedding(4,4,input_length=7000),
    Dense(32),
    Dense(16),
    Dense(1,activation='sigmoid')
])

model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])

In [10]:
history = model.fit(
    train_x, train_y, 
    epochs=50, 
    validation_data=(val_x,val_y), 
    verbose=0, 
    callbacks=[TqdmCallback(verbose=0)]
)

0epoch [00:00, ?epoch/s]

In [11]:
test_y[0]

1

In [12]:
model.evaluate(test_x, test_y)

47/47 [==============================] - 0s 2ms/step - loss: 0.6935 - accuracy: 0.4973


[0.6934535503387451, 0.4973333477973938]

In [14]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(1000)
rf.fit(train_x, train_y)

(len(test_x) - np.sum(np.abs(rf.predict(test_x) - test_y)))/len(test_x) * 100

48.06666666666667